In [9]:
import os
from dotenv import load_dotenv

from genai.extensions.langchain import LangChainInterface
from genai.schemas import GenerateParams
from genai.credentials import Credentials

from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load environment variables
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_url = os.getenv("GENAI_API", None)

# Set up credentials using the obtained API key and URL
creds = Credentials(api_key, api_endpoint=api_url)

# Define the model ID and generation parameters
model_id = "meta-llama/llama-2-70b-chat"
params = GenerateParams(
    decoding_method="greedy",
    max_new_tokens=1000,
    min_new_tokens=200,
    temperature=0.7,
)

# Create the LangChainInterface using the defined model ID, parameters, and credentials
llm = LangChainInterface(model=model_id, params=params, credentials=creds)

# Load conversation history from a file
def load_conversation_history(file_name):
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            history = file.read()
        return history
    else:
        return ""

# Save conversation history to a file
def save_conversation_history(file_name, history):
    with open(file_name, 'w') as file:
        file.write(history)

# Load conversations and store them in the vector database
file_name = 'conversation_history.txt'

# Check if the file exists and create it if not
if not os.path.exists(file_name):
    with open(file_name, 'w') as file:
        file.write('')

history = load_conversation_history(file_name)

# Check if the history is not empty before creating loaders
if history:
    loaders = [TextLoader(file_name)]
    index = VectorstoreIndexCreator(
        embedding=HuggingFaceEmbeddings(model_name='all-MiniLM-L12-v2'),
        text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    ).from_loaders(loaders)
else:
    index = None

# Set up a RetrievalQA chain using LangChainInterface, chain type, retriever, and input key
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=index.vectorstore.as_retriever() if index else None,
    input_key='question'
)

# Start the conversation loop
while True:
    user_input = input("You: ")

    if user_input.lower() == 'exit':
        print("Goodbye!")
        break

    # Update conversation history with the new user input
    history += f"User: {user_input}\n"

    # Run the chain with the user input
    response = chain.run(user_input) if index else "I'm sorry, I don't have enough data to respond."

    # Update conversation history with the bot's response
    history += f"Bot: {response}\n"

    # Save the updated conversation history to the file
    save_conversation_history(file_name, history)

    # Print the bot's response
    print("Bot:", response)


ValidationError: 1 validation error for RetrievalQA
retriever
  none is not an allowed value (type=type_error.none.not_allowed)